In [ ]:
#World Covid-19 Cases

import pandas as pd

# read the data
df = pd.read_csv("countries-aggregated.csv")
df.head()

In [ ]:
#Indonesia Covid-19 Cases

df_ind = pd.read_csv('indonesia_covid_dataset.csv')
df_ind = df_ind[['Date', 'Province', 'Cumulative_Case', 'Cumulative_Recovered', 'Cumulative_Death']]
df_ind.head()

In [ ]:
#Indonesia Province Map

import json

with open('indonesia-province.json') as file:
    indo_provinces = json.load(file)

indo_provinces["features"][0]

Choropleth Map: World Covid-19 Cases

In [ ]:
import plotly.express as px

# instantiate a figure object to build the choropleth map
fig = px.choropleth(data_frame = df, 
                    locations = 'Country', 
                    locationmode = 'country names', 
                    projection = 'natural earth', 
                    color = 'Confirmed',
                    animation_frame='Date')

# add map and colorbar titles to the figure
fig.update_layout(title_text= 'Choropleth Map of Covid-19 Confirmed Cases to Date', 
                  coloraxis_colorbar_title = 'Confirmed Cases')

# show the figure
fig.show()

Choropleth Map: Indonesia Covid-19 Cases

In [ ]:
province_id_map = {}
for feature in indo_provinces['features']:
    # add id
    feature['id'] = feature['properties']['kode']
    # map province name to its id
    province_id_map[feature['properties']['Propinsi']] = feature['id']

df_ind['id'] = df_ind['Province'].apply(lambda x: province_id_map[x])
df_ind.head()

In [ ]:
import plotly.express as px

fig = px.choropleth_mapbox(data_frame = df_ind,
                           locations = "id", # feature identifier inside data frame df_ind
                           geojson = indo_provinces, 
                           featureidkey = "id", # feature identifier inside geojson indo_provinces
                           color = "Cumulative_Case", 
                           hover_name = "Province",
                           hover_data = {'id': False, 
                                         'Cumulative_Case': True, 
                                         'Cumulative_Recovered': True, 
                                         'Cumulative_Death': True, 
                                         'Date': True},
                           mapbox_style = "carto-positron",
                           opacity = 0.7,
                           zoom = 3,
                           center = {"lat": -2.548926, "lon": 118.0148634})

fig.update_layout(title_text = 'Choropleth Map of the Latest Confirmed Covid-19 Cases in Indonesia', 
                  coloraxis_colorbar_title_text = 'Confirmed Cases', 
                  margin={"r":0,"t":50,"l":0,"b":0})

fig.show()

In [ ]:
import plotly.express as px

# instantiate a figure object to build the choropleth map
fig = px.scatter_geo(data_frame=df, 
                     locations='Country', 
                     locationmode='country names', 
                     size='Confirmed', 
                     color = 'Confirmed',
                     projection = 'natural earth',
                     hover_name = 'Country',
                     animation_frame='Date')

# add a map title and modify the map template to be dark 
fig.update_layout(template='plotly_dark', title_text = 'The Spread of Confirmed Covid-19 Cases Over Time')

# show the figure
fig.show()

In [ ]:
import geopy
from geopy.geocoders import Nominatim

# Set connection to Nominatim by specifying a unique user agent (string)
geolocator = Nominatim(user_agent='app')

# collect all the unique provinces
provinces = df_ind.Province.unique().tolist()

# define empty lists
lat = {}
lon = {}
for province in provinces:
    # fetch the province coordinate
    loc = geolocator.geocode(province + ', Indonesia')
    # if the coordinate could not be identified
    if loc is None:
        lat[province] = np.nan
        lon[province] = np.nan
    # otherwise
    else:
        lat[province] = loc.latitude
        lon[province] = loc.longitude
        
# # store the coordinate data into the data frame        
df_ind['Latitude'] = df_ind['Province'].apply(lambda x: lat[x])
df_ind['Longitude'] = df_ind['Province'].apply(lambda x: lon[x])
df_ind.head()

In [ ]:
import plotly.express as px

fig = px.scatter_mapbox(data_frame = df_ind,
                        lat = 'Latitude', 
                        lon = 'Longitude',
                        color = "Cumulative_Case", 
                        hover_name = "Province",
                        hover_data = {'Latitude': False, 
                                      'Longitude': False, 
                                      'Cumulative_Case': True, 
                                      'Cumulative_Recovered': True, 
                                      'Cumulative_Death':True, 
                                      'Date':True},
                        size = 'Cumulative_Case',
                        opacity = 0.5,
                        zoom = 3,
                        mapbox_style="carto-positron",
                        center = {"lat": -2.548926, "lon": 118.0148634})

fig.update_layout(title_text = 'Geographical Scatter Plot of the Latest Confirmed Covid-19 Cases in Indonesia', 
                  coloraxis_colorbar_title_text = 'Confirmed Cases', 
                  margin={"r":0,"t":50,"l":0,"b":0})

fig.show()

Folium

In [ ]:
df_ind_sa = df_ind.groupby('Province').max().reset_index()
df_ind_sa.head()

In [ ]:
# !pip install folium
import folium

def generatebasemap_ind(default_location =[-2.548926, 123.0148634] , default_zoom_start = 4):
    # create map of Indonesia using its latitude and longitude values
    basemap = folium.Map(location=default_location, zoom_start=default_zoom_start)
    return basemap

basemap_ind = generatebasemap_ind()
basemap_ind

In [ ]:
basemap_ind = generatebasemap_ind()

# add markers to map
for id, row in df_ind_sa.iterrows():
    label = '''<b><u>{}</u></b><br>
            Confirmed: {}<br>
            Recovered: {}<br>
            Deaths: {}<br>
            Updated: {}'''.format(row['Province'].upper(),
                                  row['Cumulative_Case'],
                                  row['Cumulative_Recovered'],
                                  row['Cumulative_Death'], 
                                  row['Date'])  
    
    label = folium.Popup(label, max_width=150)
    
    folium.Marker(
        location = [row['Latitude'], row['Longitude']],
        tooltip = '<strong>'+row['Province'].upper() + '</strong> (<i>Click for more</i>)',
        popup = label,
        icon = folium.Icon(icon='glyphicon-asterisk', color='red', size = 10),
        parse_html=False).add_to(basemap_ind)  

basemap_ind

In [ ]:
from folium.plugins import MarkerCluster

# instantiate the marker cluster object
mc = MarkerCluster()

basemap_ind = generatebasemap_ind()

# add markers to map
for id, row in df_ind_sa.iterrows():
    label = '''<b><u>{}</u></b><br>
            Confirmed: {}<br>
            Recovered: {}<br>
            Deaths: {}<br>
            Updated: {}'''.format(row['Province'].upper(),
                                  row['Cumulative_Case'],
                                  row['Cumulative_Recovered'],
                                  row['Cumulative_Death'], 
                                  row['Date'])  
    
    label = folium.Popup(label, max_width=150)
    
    mc.add_child(folium.Marker(location = [row['Latitude'], row['Longitude']],
                               tooltip = '<strong>'+row['Province'].upper() + '</strong> (<i>Click for more</i>)',
                               popup = label,
                               icon = folium.Icon(icon='glyphicon-asterisk', color='red', size = 10),
                               parse_html=False))

# Assign the marker cluster to the basemap
basemap_ind.add_child(mc)

basemap_ind

In [ ]:
from folium.plugins import HeatMap

# define the basemap
basemap_ind = generatebasemap_ind()

# Configure the heatmap class and add it to the basemap
HeatMap(data=df_ind_sa[['Latitude', 'Longitude', 'Cumulative_Case']],
        max_zoom = 20, 
        radius=13).add_to(basemap_ind)



# add markers to map
for id, row in df_ind_sa.iterrows():
    label = '''<b><u>{}</u></b><br>
            Confirmed: {}<br>
            Recovered: {}<br>
            Deaths: {}<br>
            Updated: {}'''.format(row['Province'].upper(),
                                  row['Cumulative_Case'],
                                  row['Cumulative_Recovered'],
                                  row['Cumulative_Death'], 
                                  row['Date'])  
    
    # create a Popup instance
    label = folium.Popup(label, max_width=150)
    
    # use circle marker instead of the regular marker
    folium.CircleMarker(location = [row['Latitude'], row['Longitude']],
                        tooltip = '<strong>'+row['Province'].upper() + '</strong> (<i>Click for more</i>)',
                        radius = 1, 
                        color = 'blue',
                        popup = label,
                        fill = True,
                        opacity = 0.1,
                        fill_opacity = 0.1,
                        parse_html = False).add_to(basemap_ind)  

basemap_ind